In [1]:
# ==================================================================================
# Diplomado en Estadística Aplicada a la Toma de Decisiones con Lenguaje R y Python
# Universidad Privada Boliviana
# ----------------------------------------------------------------------------------
#   MODELOS PREDICTIVOS PARA LA TOMA DE DECISIONES ESTRATEGICAS
# ----------------------------------------------------------------------------------
#         Enrique Alejandro Laurel Cossio, Mayo 2025
# ==================================================================================
#                Introducción a Redes Neuronales
# ==================================================================================

In [20]:
# Cargamos Librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,mean_squared_error,r2_score,confusion_matrix,recall_score,classification_report,precision_score
from sklearn.metrics import roc_curve, roc_auc_score

from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import GridSearchCV

# redes neuronales
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization
from keras.regularizers import l2
from tensorflow.random import set_seed
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import random

# Definimos la semilla aleatoria para replicabilidad del modelo
semilla=777

set_seed(semilla)
# semilla en NumPy
np.random.seed(semilla)
# semilla en Python (random)
random.seed(semilla)
# semilla en TensorFlow
tf.random.set_seed(semilla)

# 1. REDES NEURONALES - REGRESIÓN

In [12]:
# cargamos datos
url='https://raw.githubusercontent.com/ealaurel/MODELOS_PREDICTIVOS/main/data/insurance.csv'
seguros = pd.read_csv(url,sep=',', encoding='iso-8859-1')

print(seguros.shape) #
seguros.head(2)

(1338, 7)


,age,sex,bmi,children,smoker,region,charges
0,19,female,27.90,0,yes,southwest,16884.9240
1,18,male,33.77,1,no,southeast,1725.5523


In [13]:
# una copia de la fuente de datos
df = seguros.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [14]:
# a dummies las variables categoricas
df = pd.get_dummies(df, columns=['region','sex','smoker'], drop_first=True)
print(df.shape)
df.head(2)

(1338, 9)


,age,bmi,children,charges,region_northwest,region_southeast,region_southwest,sex_male,smoker_yes
0,19,27.90,0,16884.9240,False,False,True,False,True
1,18,33.77,1,1725.5523,False,True,False,True,False


In [15]:
# variables dummies  a entero
df = df.replace({True: 1, False: 0})

<ipython-input-15-fd397598af70>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({True: 1, False: 0})


In [16]:
# crear el campo logaritmo de charges(costo de seguro medico)
df['log_charges'] = np.log(df['charges'])

In [17]:
# Definir las variables independientes y dependientes
X = df[['age', 'sex_male', 'bmi', 'children', 'smoker_yes', 'region_northwest','region_southeast','region_southwest']] # variables explicativas
y = df['log_charges'] # variables dependiente

# Añadir una constante a las variables independientes
X = sm.add_constant(X) # relacionado intercepto

In [18]:
# División de conjunto de datos en entrenamiento y testeo
# train - test
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 22) #Separamos 30% para test

In [19]:
# Crear el modelo
model_RNA_Regres = Sequential()
model_RNA_Regres.add(Dense(units=64, activation='relu', input_dim=8))  # Capa de entrada
model_RNA_Regres.add(BatchNormalization())
model_RNA_Regres.add(Dense(units=32, activation='relu'))  # Primera capa oculta
model_RNA_Regres.add(BatchNormalization())
model_RNA_Regres.add(Dense(units=16, activation='relu'))  # Segunda capa oculta
model_RNA_Regres.add(BatchNormalization())
model_RNA_Regres.add(Dense(units=1, activation='linear'))  # Capa de salida (regresion)

# Compilar el modelo
model_RNA_Regres.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

# Entrenar el modelo
model_RNA_Regres.fit(x_train.drop(columns=['const']), y_train, epochs=10, batch_size=32)

# Evaluar el rendimiento del modelo
loss, mae = model_RNA_Regres.evaluate(x_train.drop(columns=['const']), y_train)
print(f"Pérdida: {loss:.4f}, Error Cuadratico Medio: {mae:.4f}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 82.0856 - mean_squared_error: 82.0856
Epoch 2/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 78.2566 - mean_squared_error: 78.2566
Epoch 3/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 74.2622 - mean_squared_error: 74.2622
Epoch 4/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 69.4499 - mean_squared_error: 69.4499
Epoch 5/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 63.8719 - mean_squared_error: 63.8719
Epoch 6/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 57.6704 - mean_squared_error: 57.6704
Epoch 7/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 50.9844 - mean_squared_error: 50.9844
Epoch 8/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 44.0914 - mean_squared_error: 44.0914
Epoch 9/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 37.2555 - mean_squared_error: 37.2555
Epoch 10/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 30.6844 - mean_squared_error: 30.6844
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 

In [21]:
# guardamos el mejor modelo entrenado
early_stopping = EarlyStopping(monitor='mean_squared_error', patience=5, mode='min', restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='mean_squared_error', save_best_only=True, mode='min')


# Entrenar el modelo con los callbacks
history = model_RNA_Regres.fit(
    x_train.drop(columns=['const']),
    y_train,
    epochs=100, # Incrementa las epochs para dar tiempo a EarlyStopping
    batch_size=32,
    callbacks=[early_stopping, model_checkpoint]
)

# Cargar el mejor modelo guardado
best_model = tf.keras.models.load_model('best_model.h5')

# Evaluar el rendimiento del mejor modelo en los datos de entrenamiento
loss, mae = best_model.evaluate(x_train.drop(columns=['const']), y_train)
print(f"Pérdida del mejor modelo (Entrenamiento): {loss:.4f}, Error Cuadratico Medio: {mae:.4f}")

# Opcional: Evaluar el mejor modelo en los datos de prueba
loss_test, mae_test = best_model.evaluate(x_test.drop(columns=['const']), y_test)
print(f"Pérdida del mejor modelo (Prueba): {loss_test:.4f}, Error Cuadratico Medio: {mae_test:.4f}")


Epoch 1/100
17/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 25.0160 - mean_squared_error: 25.0160   

30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 24.5947 - mean_squared_error: 24.5947
Epoch 2/100
19/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 19.4586 - mean_squared_error: 19.4586 

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 19.1349 - mean_squared_error: 19.1349
Epoch 3/100
17/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 14.7193 - mean_squared_error: 14.7193  

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 14.4122 - mean_squared_error: 14.4122
Epoch 4/100
20/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 10.6778 - mean_squared_error: 10.6778 

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 10.4854 - mean_squared_error: 10.4854
Epoch 5/100
29/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 7.4464 - mean_squared_error: 7.4464

...

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.1191 - mean_squared_error: 0.1191
Epoch 31/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1254 - mean_squared_error: 0.1254
Epoch 32/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1200 - mean_squared_error: 0.1200
Epoch 33/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1315 - mean_squared_error: 0.1315
Epoch 34/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1201 - mean_squared_error: 0.1201
Epoch 35/100
19/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1108 - mean_squared_error: 0.1108 

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1146 - mean_squared_error: 0.1146
Epoch 36/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.1178 - mean_squared_error: 0.1178
Epoch 37/100
20/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1072 - mean_squared_error: 0.1072   

30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.1107 - mean_squared_error: 0.1107
Epoch 38/100
18/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1085 - mean_squared_error: 0.1085  

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.1121 - mean_squared_error: 0.1121
Epoch 39/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1129 - mean_squared_error: 0.1129
Epoch 40/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1238 - mean_squared_error: 0.1238
Epoch 41/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1229 - mean_squared_error: 0.1229
Epoch 42/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1243 - mean_squared_error: 0.1243
Epoch 43/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1301 - mean_squared_error: 0.1301


30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2042 - mean_squared_error: 0.2042
Pérdida del mejor modelo (Entrenamiento): 0.2252, Error Cuadratico Medio: 0.2252
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2778 - mean_squared_error: 0.2778 
Pérdida del mejor modelo (Prueba): 0.3110, Error Cuadratico Medio: 0.3110


In [22]:
#cargar el mejor modelo que se haya guardado

# Cargar el mejor modelo guardado
best_model = load_model('best_model.h5')

best_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 16)             │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,651 (14.27 KB)

 Trainable params: 3,425 (13.38 KB)

 Non-trainable params: 224 (896.00 B)

 Optimizer params: 2 (12.00 B)

In [23]:
# Evaluar el rendimiento del modelo
loss, mae = best_model.evaluate(x_train.drop(columns=['const']), y_train)
print(f"Pérdida: {loss:.4f}, Error Cuadratico Medio: {mae:.4f}")

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2042 - mean_squared_error: 0.2042
Pérdida: 0.2252, Error Cuadratico Medio: 0.2252


In [24]:
# Ajustar el modelo de regresión lineal
modelo_OLS = sm.OLS(y_train, x_train).fit()

# Resumen del modelo
print(modelo_OLS.summary())

                            OLS Regression Results                            
Dep. Variable:            log_charges   R-squared:                       0.776
Model:                            OLS   Adj. R-squared:                  0.774
Method:                 Least Squares   F-statistic:                     402.2
Date:                Fri, 30 May 2025   Prob (F-statistic):          2.63e-295
Time:                        23:25:34   Log-Likelihood:                -547.15
No. Observations:                 936   AIC:                             1112.
Df Residuals:                     927   BIC:                             1156.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                6.9352      0.084  

In [25]:
# ajustamos el indice en la data de test
x_test.reset_index(drop=True, inplace=True)

In [26]:
# calculamos predicción

# Regresión Lineal (Mínimos cuadrados ordinarios)
print('Predicción Regresión Lineal - OLS')
y_pred_OLS = modelo_OLS.predict(x_test)
print(y_pred_OLS.head(2))

# RNA
print('Predicción RNA - Regresión')
y_pred_RNA = best_model.predict(x_test.drop(columns='const'))
y_pred_RNA = pd.DataFrame(y_pred_RNA)
print(y_pred_RNA.head(2))

Predicción Regresión Lineal - OLS
0    9.412384
1    9.694925
dtype: float64
Predicción RNA - Regresión
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
          0
0  9.871866
1  9.801596


In [27]:
# Error Cuadratico Medio

# Regresión Lineal - OLS
mse_OLS = mean_squared_error(y_test, y_pred_OLS)
print(f"Error cuadrático medio (MSE)- Regresión Lineal OLS: {mse_OLS:.4f}")

# RNA - Regresión
mse_RNA = mean_squared_error(y_test, y_pred_RNA)
print(f"Error cuadrático medio (MSE)- RNA: {mse_RNA:.4f}")
# Mientras menos mejor

Error cuadrático medio (MSE)- Regresión Lineal OLS: 0.2156
Error cuadrático medio (MSE)- RNA: 0.3110


# 2. REDES NEURONALES - CLASIFICACIÓN

In [28]:
# cargamos datos
url='https://raw.githubusercontent.com/ealaurel/MODELOS_PREDICTIVOS/main/data/ifood_df.csv'
ifood_df = pd.read_csv(url,sep=',', encoding='iso-8859-1')

print(ifood_df.shape) #
ifood_df.head(2)

(2205, 39)


,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,...,marital_Together,marital_Widow,education_2n Cycle,education_Basic,education_Graduation,education_Master,education_PhD,MntTotal,MntRegularProds,AcceptedCmpOverall
0,58138.0,0,0,58,635,88,546,172,88,88,...,0,0,0,0,1,0,0,1529,1441,0
1,46344.0,1,1,38,11,1,6,2,1,6,...,0,0,0,0,1,0,0,21,15,0


In [29]:
# una copia a los datos
df = ifood_df.copy()

In [30]:
# seleccionamos variables explicativas y variables dependiente
X = df[['Kidhome','Recency','education_Basic','MntRegularProds']]
y = df['Response']

In [ ]:
# definimos una semilla
#semilla = 777

In [31]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=semilla)

In [32]:
# reseteamnos indices
X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

In [33]:
# Entrenamiento Modelos de Redes Neuronales Artificiales

# Crear el modelo
model_RNA_Clasif = Sequential()
model_RNA_Clasif.add(Dense(units=64, activation='relu', input_dim=4,kernel_regularizer=l2(0.01)))  # Capa de entrada
model_RNA_Clasif.add(Dense(units=32, activation='relu',kernel_regularizer=l2(0.01)))  # Primera capa oculta
model_RNA_Clasif.add(Dense(units=16, activation='relu',kernel_regularizer=l2(0.01)))  # Segunda capa oculta
model_RNA_Clasif.add(BatchNormalization())
model_RNA_Clasif.add(Dense(units=1, activation='sigmoid'))  # Capa de salida (clasificación binaria)

# Compilar el modelo
model_RNA_Clasif.compile(loss='binary_crossentropy', optimizer='adam', metrics=['precision'])

# Entrenar el modelo
model_RNA_Clasif.fit(X_train, y_train, epochs=10, batch_size=32)

# Evaluar el rendimiento del modelo
loss, precision = model_RNA_Clasif.evaluate(X_train, y_train)
print(f"Pérdida: {loss:.4f}, Precision: {precision:.4f}")

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 1.5212 - precision: 0.0951
Epoch 2/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0678 - precision: 0.1534
Epoch 3/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.8485 - precision: 0.2225
Epoch 4/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.7143 - precision: 0.2992
Epoch 5/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6475 - precision: 0.4696
Epoch 6/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.6077 - precision: 0.8421
Epoch 7/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.5883 - precision: 0.0000e+00
Epoch 8/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.5603 - precision: 0.0000e+00
Epoch 9/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5572 - precision: 0.0000e+00
Epoch 10/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5469 - precision: 0.0000e+00
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5515 - precision: 0.0000e+00
Pérdida: 0.5511, Precision: 0.0000


In [34]:
#  guardamos el mejor modelo

early_stopping_clasif = EarlyStopping(monitor='precision', patience=10, mode='max', restore_best_weights=True)
model_checkpoint_clasif = ModelCheckpoint('best_classification_model.h5', monitor='precision', save_best_only=True, mode='max')

# Entrenar el modelo de clasificación
history_clasif = model_RNA_Clasif.fit(
    X_train,
    y_train,
    epochs=200, # Incrementa las epochs para dar tiempo a EarlyStopping
    batch_size=32,
    callbacks=[early_stopping_clasif, model_checkpoint_clasif]
)

# Cargar el mejor modelo de clasificación guardado
best_classification_model = load_model('best_classification_model.h5')

# Imprimir un resumen del modelo cargado para verificar
best_classification_model.summary()

# Evaluar el rendimiento del mejor modelo de clasificación en los datos de entrenamiento
loss_clasif, precision_clasif = best_classification_model.evaluate(X_train, y_train)
print(f"Pérdida del mejor modelo de clasificación (Entrenamiento): {loss_clasif:.4f}, Precision: {precision_clasif:.4f}")

# Opcional: Evaluar el mejor modelo de clasificación en los datos de prueba
loss_test_clasif, precision_test_clasif = best_classification_model.evaluate(X_test, y_test)
print(f"Pérdida del mejor modelo de clasificación (Prueba): {loss_test_clasif:.4f}, Precision: {precision_test_clasif:.4f}")

Epoch 1/200
41/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5290 - precision: 0.0000e+00

56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5317 - precision: 0.0000e+00
Epoch 2/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5245 - precision: 0.0000e+00
Epoch 3/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5115 - precision: 0.0000e+00
Epoch 4/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5045 - precision: 0.0000e+00
Epoch 5/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4942 - precision: 0.0000e+00
Epoch 6/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4907 - precision: 0.0000e+00
Epoch 7/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4893 - precision: 0.0000e+00
Epoch 8/200
51/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4730 - precision: 0.2353    

56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.4736 - precision: 0.3158
Epoch 9/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4616 - precision: 0.0000e+00
Epoch 10/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4497 - precision: 0.0000e+00
Epoch 11/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4614 - precision: 0.0000e+00
Epoch 12/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4445 - precision: 0.0000e+00
Epoch 13/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4556 - precision: 0.0000e+00
Epoch 14/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4535 - precision: 0.0000e+00
Epoch 15/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.4344 - precision: 0.0000e+00
Epoch 16/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4347 - precision: 0.0000e+00
Epoch 17/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4528 - precision: 0.0000e+00
Epoch 18/200
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4296 - precision: 0.0000e+00


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 64)             │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 16)             │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,011 (11.77 KB)

 Trainable params: 2,977 (11.63 KB)

 Non-trainable params: 32 (128.00 B)

 Optimizer params: 2 (12.00 B)

56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4827 - precision_1: 0.0000e+00
Pérdida del mejor modelo de clasificación (Entrenamiento): 0.4831, Precision: 0.0000
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5294 - precision_1: 0.0000e+00 
Pérdida del mejor modelo de clasificación (Prueba): 0.5492, Precision: 0.0000


In [35]:
# Árboles de Decisión
# hallamos los mejores hiperparametros del modelo
# Crear el clasificador
clf = DecisionTreeClassifier()

# Definir la grilla de hiperparámetros a explorar
param_grid = {
    'max_depth': list(range(1,20)),#[1, 2,3,4,5,6,7,8,9,10,15, ,20],
    'min_samples_split': list(range(1,10)),
    'min_samples_leaf': list(range(1,4))
}

# Realizar búsqueda en la grilla
grid_search = GridSearchCV(clf, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Mejores hiperparámetros encontrados
best_params = grid_search.best_params_
best_params

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
285 fits failed out of a total of 2565.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
285 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.11/dist-packages/sklearn/uti

{'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2}

In [36]:
# Entrenamos el Modelo de Árbol de Decisión para Clasificación

# Crea un clasificador de árbol de decisión
modelo_ArbolDecisionClasif = DecisionTreeClassifier(max_depth=3,min_samples_leaf=1,min_samples_split=2,random_state=semilla)

# Entrena el modelo en los datos de entrenamiento
modelo_ArbolDecisionClasif.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=3, random_state=777)

In [ ]:
# ajustamos el indice en x_test
#X_test.reset_index(drop=True, inplace=True)

In [37]:
# Realiza predicciones en el conjunto de datos de prueba

# RNA
print('Predicción RNA')
y_pred_RNA_Clasif = best_classification_model.predict(X_test)
y_pred_RNA_Clasif = pd.DataFrame(y_pred_RNA_Clasif)
print(y_pred_RNA_Clasif.head(2))

# Arbol de Decisión - Clasificación
print('Predicción Árbol de Decisión - Clasificación')
y_pred_ArbolDecisionClasif = modelo_ArbolDecisionClasif.predict_proba(X_test)
y_pred_ArbolDecisionClasif = pd.DataFrame(y_pred_ArbolDecisionClasif)
y_pred_ArbolDecisionClasif = y_pred_ArbolDecisionClasif.iloc[:,1]
print(y_pred_ArbolDecisionClasif.head(2))

Predicción RNA
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
          0
0  0.141468
1  0.022798
Predicción Árbol de Decisión - Clasificación
0    0.076233
1    0.015504
Name: 1, dtype: float64


In [38]:
# Obteniendo el mejor corte de predicción
# Calcular la curva ROC - RNA
fpr, tpr, thresholds = roc_curve(y_test, y_pred_RNA_Clasif)

# Encontrar el umbral óptimo (máximo índice de Youden)
youden_index = tpr - fpr
optimal_threshold_RNA = thresholds[np.argmax(youden_index)]

print("Umbral óptimo RNA:", optimal_threshold_RNA)

Umbral óptimo RNA: 0.21783029


In [40]:
# Obteniendo el mejor corte de predicción
# Calcular la curva ROC - Arbol de Decisión
fpr, tpr, thresholds = roc_curve(y_test, y_pred_ArbolDecisionClasif)

# Encontrar el umbral óptimo (máximo índice de Youden)
youden_index = tpr - fpr
optimal_threshold_AD = thresholds[np.argmax(youden_index)]

print("Umbral óptimo- Arbol de Decisión:", optimal_threshold_AD)

Umbral óptimo- Arbol de Decisión: 0.20512820512820512


In [41]:
# Matriz de confusión

            #Predicción
            #   0   1
#Valor Real 0 # VN FP
#Valor Real 1 # FN VP

# Regresión Logistica - RNA
confusion = confusion_matrix(y_test, (y_pred_RNA_Clasif>=optimal_threshold_RNA).astype(int))
print("Matriz de confusión - RNA:")
print(confusion)

# Árbol de Decisión - Clasificación
confusion = confusion_matrix(y_test, (y_pred_ArbolDecisionClasif>=optimal_threshold_AD).astype(int))
print("\nMatriz de confusión - Árbol de Decisión:")
print(confusion)

Matriz de confusión - RNA:
[[202 160]
 [ 20  59]]

Matriz de confusión - Árbol de Decisión:
[[256 106]
 [ 33  46]]


In [42]:
# Exactitud o Accuracy del modelo

# RNA
exactitud_test_RNA = accuracy_score(y_test, (y_pred_RNA_Clasif>=optimal_threshold_RNA).astype(int))
print("Exactitud del modelo-RNA:", exactitud_test_RNA)

# Árbol de Decisión - Clasificación
exactitud_test_ArbolDecisionClasif = accuracy_score(y_test, (y_pred_ArbolDecisionClasif>=optimal_threshold_AD).astype(int))
print("Exactitud del modelo- Árboles de Decisión:", exactitud_test_ArbolDecisionClasif)

Exactitud del modelo-RNA: 0.5918367346938775
Exactitud del modelo- Árboles de Decisión: 0.6848072562358276


In [43]:
# Sensibilidad (Proporción de verdaderos positivos de los valores reales positivos)

# RNA
sensibilidad_test_RNA = recall_score(y_test, (y_pred_RNA_Clasif>=optimal_threshold_RNA).astype(int))
print("Sensibilidad - RNA:", sensibilidad_test_RNA)

# Árbol de Decisión - Clasificación
sensibilidad_test_ArbolDecisionClasif = recall_score(y_test, (y_pred_ArbolDecisionClasif>=optimal_threshold_AD).astype(int))
print("Sensibilidad - Arbol de Decisión:", sensibilidad_test_ArbolDecisionClasif)

Sensibilidad - RNA: 0.7468354430379747
Sensibilidad - Arbol de Decisión: 0.5822784810126582


In [44]:
# Precisión (Proporción de Verdaderos Positivos de las predicciónes positivas del modelo)

# RNA
precision_score_RNA = precision_score(y_test, (y_pred_RNA_Clasif>=optimal_threshold_RNA).astype(int))
print("Precisión del modelo - RNA", precision_score_RNA)

# Arbol de Decisión - Clasificación
precision_score_ArbolDecisionClasif = precision_score(y_test, (y_pred_ArbolDecisionClasif>=optimal_threshold_AD).astype(int))
print("Precisión del modelo - Árbol de Decisión", precision_score_ArbolDecisionClasif)

Precisión del modelo - RNA 0.2694063926940639
Precisión del modelo - Árbol de Decisión 0.3026315789473684


# 3. RESULTADOS

In [ ]:
# Objetivo: tener una mayor tasa de exito en las campañas de marketing y benfecios positivos

In [45]:
print('Nro de clientes en el piloto',df.shape[0])

Nro de clientes en el piloto 2205


In [46]:
costo_piloto=6720
costo_piloto

6720

In [47]:
ingreso_piloto=3674
ingreso_piloto

3674

In [48]:
# beneficio del piloto
ingreso_piloto-costo_piloto

-3046

In [ ]:
# Propuesta

In [49]:
# calculamos el ingreso por cliente
print('Clientes que aceptaron la oferta',df[df['Response']==1].shape[0],'de',df.shape[0],'clientes')
ingreso_xcliente = ingreso_piloto/df[df['Response']==1].shape[0]
print('ingreso por cliente que aceptó la oferta',ingreso_xcliente)

Clientes que aceptaron la oferta 333 de 2205 clientes
ingreso por cliente que aceptó la oferta 11.033033033033034


In [50]:
# costo por cliente
costo_xcliente = costo_piloto/df.shape[0]
print('Costo por cliente',costo_xcliente)

Costo por cliente 3.0476190476190474


In [51]:
# RNA
confusion_RNA = confusion_matrix(y_test, (y_pred_RNA_Clasif>=optimal_threshold_RNA).astype(int))
print("Matriz de confusión - RNA:")
print(confusion_RNA)

# Extrae los valores
VN_RNA,  FP_RNA,FN_RNA, VP_RNA = confusion_RNA.ravel()

print(f"Verdaderos Negativos (VN): {VN_RNA}")
print(f"Falsos Positivos (FP): {FP_RNA}")
print(f"Falsos Negativos (FN): {FN_RNA}")
print(f"Verdaderos Positivos (VP): {VP_RNA}")

Matriz de confusión - RNA:
[[202 160]
 [ 20  59]]
Verdaderos Negativos (VN): 202
Falsos Positivos (FP): 160
Falsos Negativos (FN): 20
Verdaderos Positivos (VP): 59


In [52]:
# Árbol de Decisión - Clasificación
confusion_DT = confusion_matrix(y_test, (y_pred_ArbolDecisionClasif>=optimal_threshold_AD).astype(int))
print("\nMatriz de confusión - Árbol de Decisión:")
print(confusion_DT)

# Extrae los valores
VN_DT, FP_DT, FN_DT, VP_DT = confusion_DT.ravel()

print(f"Verdaderos Negativos (VN): {VN_DT}")
print(f"Falsos Positivos (FP): {FP_DT}")
print(f"Falsos Negativos (FN): {FN_DT}")
print(f"Verdaderos Positivos (VP): {VP_DT}")


Matriz de confusión - Árbol de Decisión:
[[256 106]
 [ 33  46]]
Verdaderos Negativos (VN): 256
Falsos Positivos (FP): 106
Falsos Negativos (FN): 33
Verdaderos Positivos (VP): 46


In [53]:
# RNA
# inversión o costo - RNA, datos de testeo, representan un 20% del total de clientes en piloto

# numero de clientes con alta probabilidad
cli_test_alta_prob_RNA = FP_RNA+VP_RNA
print('Clientes Test con alta probabilidad',cli_test_alta_prob_RNA)

# beneficio esperado en clientes de datos de test con alta probabilidad
benef_cli_test_alta_prob_RNA=ingreso_xcliente*VP_RNA-costo_xcliente*(FP_RNA+VP_RNA)
print('Beneficio Cli_test con alta probabilidad($us)',benef_cli_test_alta_prob_RNA)

Clientes Test con alta probabilidad 219
Beneficio Cli_test con alta probabilidad($us) -16.479622479622435


In [54]:
# proporcion de clientes que se enviara la campaña
#(Predicción Positiva entre el total de clientes)
prop_campania_RNA=(VP_RNA+FP_RNA)/(VP_RNA+FP_RNA+VN_RNA+FN_RNA)
prop_campania_RNA

np.float64(0.4965986394557823)

In [55]:
# bajo el supuesto que el piloto es una muestra representativa de la población
# bajo el supuesto que la poblacion de clientes es 2,000,000
pob_clientes=2000000
print('Población de Clientes',pob_clientes)

# Se envia la campaña de marketing unicamente al 70% de la población que calificaran como muy probables a aceptar la oferta
cli_campania_RNA = pob_clientes*prop_campania_RNA
print('Clientes que recibiran la campaña de marketing',cli_campania_RNA)

Población de Clientes 2000000
Clientes que recibiran la campaña de marketing 993197.2789115646


In [56]:
# Beneficios Esperados con modelo RNA
# usando la siguiente formula
# donde :
# N: población calificable para la campaña (cli_campania_RNA, clientes con alta probabilidad calculada por el modelo)
# n: clientes calificables en el subconjunto de testeo,muestra de la población(cli_test_alta_prob_RNA, con alta probabilidad)
# Factor de expansión = N/n
# Benef_test: Beneficio en el grupo de testeo (benef_cli_test_alta_prob_RNA)
# Asi el beneficio esperado en la población calificable para la campaña es:  Benef_test* (N/n)
print('Beneficio esperado de la campaña en población calificada(alta probabilidad), $us',(benef_cli_test_alta_prob_RNA)*(cli_campania_RNA/cli_test_alta_prob_RNA))

Beneficio esperado de la campaña en población calificada(alta probabilidad), $us -74737.51691438747


Vista Arboles de Decisión

In [57]:
# DT
# inversión o costo - DT, datos de testeo, representan un 20% del total de clientes en piloto

# numero de clientes con alta probabilidad
cli_test_alta_prob_DT = FP_DT+VP_DT
print('Clientes Test con alta probabilidad',cli_test_alta_prob_DT)

# beneficio esperado en clientes de datos de test con alta probabilidad
benef_cli_test_alta_prob_DT=ingreso_xcliente*VP_DT-costo_xcliente*(FP_DT+VP_DT)
print('Beneficio Cli_test con alta probabilidad($us)',benef_cli_test_alta_prob_DT)

Clientes Test con alta probabilidad 152
Beneficio Cli_test con alta probabilidad($us) 44.28142428142439


In [58]:
# proporcion de clientes que se enviara la campaña
#(Predicción Positiva entre el total de clientes)
prop_campania_DT=(VP_DT+FP_DT)/(VP_DT+FP_DT+VN_DT+FN_DT)
prop_campania_DT

np.float64(0.34467120181405897)

In [59]:
# bajo el supuesto que el piloto es una muestra representativa de la población
# bajo el supuesto que la poblacion de clientes es 2,000,000
pob_clientes=2000000
print('Población de Clientes',pob_clientes)

# Se envia la campaña de marketing unicamente al 70% de la población que calificaran como muy probables a aceptar la oferta
cli_campania_DT = pob_clientes*prop_campania_DT
print('Clientes que recibiran la campaña de marketing',cli_campania_DT)

Población de Clientes 2000000
Clientes que recibiran la campaña de marketing 689342.403628118


In [60]:
# Beneficios Esperados con modelo RNA
# usando la siguiente formula
# donde :
# N: población calificable para la campaña (cli_campania_DT, clientes con alta probabilidad calculada por el modelo)
# n: clientes calificables en el subconjunto de testeo,muestra de la población(cli_test_alta_prob_DT, con alta probabilidad)
# Factor de expansión = N/n
# Benef_test: Beneficio en el grupo de testeo (benef_cli_test_alta_prob_DT)
# Asi el beneficio esperado en la población calificable para la campaña es:  Benef_test* (N/n)
print('Beneficio esperado de la campaña en población calificada(alta probabilidad), $us',(benef_cli_test_alta_prob_DT)*(cli_campania_DT/cli_test_alta_prob_DT))

Beneficio esperado de la campaña en población calificada(alta probabilidad), $us 200822.78585679998


- Problema y objetivo del negocio
- Exploración de datos, como la variable respuesta
- Modelo entrenado
- Resultados del modelo
- Propuesta de negocio en base al modelo

- Codigo del modelo.

Entregables:
1. Presentación
2. Codigo